This analysis looks at burn severity of wildfires in the Sierra Nevada region from 2012 to 2021. 

 
- Not all wildfires were documented in the data sets
- The index is used to classify burn severity as unchanged, low, moderate or high severity. >> impacts not intensity
    - Unchanged: Areas indistinguishable from pre-fire conditions. This does not necessarily indicate that the area was unburned
    - Low: Areas of surface fire with little change in cover and little mortality of the structurally dominant vegetation
    - Moderate: Areas between low and high severity, with a mixture of effects on the structurally dominant vegetation
    - High: Areas where the dominant vegetation has high to complete mortality
    
More info on calculating area size by TIFF class
- https://gis.stackexchange.com/questions/199113/how-to-calculate-raster-area-or-number-of-cells-for-multiple-classes-in-a-raster?noredirect=1&lq=1
- https://gis.stackexchange.com/questions/277126/what-are-the-methods-for-calculating-area-from-a-raster-in-qgis

### Questions
- How much has the area burned at high severity increased since 2012? 
- Has % of area burned at the highest severity also increased? 
- Impacts of the increasing areas with high burn index?
    - tree canopy?
    - animal habitats? 
    - property loss? 
    - ...
- Composite Burn Index (CBI) Data for the Conterminous US, [Collected Between 1996 and 2018](https://www.sciencebase.gov/catalog/item/5d963705e4b0c4f70d110ee6)

In [1]:
# import sys
# !{sys.executable} -m pip install vega

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from osgeo import gdal
import rasterio
import altair as alt
import altair_latimes as lat
alt.renderers.enable('altair_viewer', inline=True)

RendererRegistry.enable('altair_viewer')

In [3]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

In [4]:
def analyze_index(file):
    raw = pd.read_csv(file, index_col = 0)
    raw['percent_of_total'] = 0
    raw['total'] = 0
    df = raw.copy()
    grouped = df.groupby('year').sqm.sum().reset_index()
    grouped['acre'] = grouped.sqm.apply(lambda x: x*0.000247105)
    df['acre'] = df.sqm.apply(lambda x: x*0.000247105)
    for i in range(len(grouped)):
        for k in range(len(df)):
            if grouped.year[i] == df.year[k]:
                df['percent_of_total'].iloc[k] = df.acre[k] / grouped.acre[i]
                df['total'].iloc[k] = grouped.acre[i]
    return grouped, df

# CBI-4 (burn severity index)

In [5]:
cbi4_by_year, cbi4_by_index = analyze_index('output/sum_of_sierra_fire_size_by_burn_index.csv')

/var/folders/zv/3yvy_fbd0mz2lvqqtd13wypc0000gp/T/ipykernel_38874/1028497440.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['percent_of_total'].iloc[k] = df.acre[k] / grouped.acre[i]
/var/folders/zv/3yvy_fbd0mz2lvqqtd13wypc0000gp/T/ipykernel_38874/1028497440.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total'].iloc[k] = grouped.acre[i]


In [6]:
alt.Chart(cbi4_by_year, title="Total area burned by year").mark_bar(size=20).encode(
    x=alt.X(
        "year",
        title="Year",
        axis=alt.Axis(format=''),
        scale=alt.Scale(domain=[2012, 2021])
    ),
    y=alt.Y(
        'acre',
        title='Acres',
        axis=alt.Axis(format=""),
    ),
    tooltip=['year', 'acre']
).properties(width=600)

alt.Chart(...)

In [7]:
total_burn_after20 = cbi4_by_year.iloc[8:].sqm.sum()
total_burn_before20 = cbi4_by_year.iloc[:8].sqm.sum()

In [8]:
total_burn_after20 / total_burn_before20

1.7749369360596021

- The total area burned after 2020 was 1.8 times the total area burned from 2012 to 2019. 
- The total area burned in 2021 was 16.7 times that of 2012.

In [9]:
high_severity = cbi4_by_index[cbi4_by_index.ind == 4].reset_index(drop=True)

alt.Chart(high_severity, title="Total area burned at the highest severity by year").mark_bar(size=20).encode(
    x=alt.X(
        "year",
        title="Year",
        axis=alt.Axis(format=''),
        scale=alt.Scale(domain=[2012, 2021])
    ),
    y=alt.Y(
        'acre',
        title='Acres',
        axis=alt.Axis(format=""),
    ),
    tooltip=['year', 'acre']
).properties(width=600)

alt.Chart(...)

In [10]:
hsevere_after20 = high_severity.iloc[8:].sqm.sum()
hsevere_before20 = high_severity.iloc[:8].sqm.sum()

In [11]:
hsevere_after20 / hsevere_before20

2.341756803020329

- The total area burned at high severity after 2020 was 2.3 times that of 2012 to 2019.
- In 2021, the total area burned at the highest severity level was 36 times that of 2012.

In [12]:
alt.Chart(high_severity, title="Percentage of total area burned at the highest severity by year").mark_bar(size=20).encode(
    x=alt.X(
        "year",
        title="Year",
        axis=alt.Axis(format=''),
        scale=alt.Scale(domain=[2012, 2021])
    ),
    y=alt.Y(
        'percent_of_total',
        title='',
        axis=alt.Axis(format=".0%"),
    ),
    tooltip=['year', 'percent_of_total']
).properties(width=600)

alt.Chart(...)

In [13]:
high_severity.to_csv("output/sierra_nevada_high_severity_size_by_year.csv")

In [14]:
high_severity

,year,ind,ind_name,sqm,percent_of_total,total,acre
0,2012,4,high,5.737770e+07,0.208463,6.801358e+04,14178.316559
1,2013,4,high,4.188924e+08,0.330357,3.133289e+05,103510.406502
2,2014,4,high,2.159028e+08,0.445007,1.198873e+05,53350.661394
3,2015,4,high,2.473317e+08,0.381960,1.600088e+05,61116.899729
4,2016,4,high,1.986111e+08,0.521604,9.409022e+04,49077.795866
5,2017,4,high,9.700200e+07,0.257754,9.299426e+04,23969.679210
6,2018,4,high,2.366775e+08,0.258450,2.262884e+05,58484.193638
7,2019,4,high,1.291959e+08,0.394467,8.093180e+04,31924.952870
8,2020,4,high,1.689453e+09,0.456106,9.152957e+05,417472.283565
9,2021,4,high,2.059679e+09,0.448136,1.135721e+06,508956.929874


The % of fire burned at the highest severity in Sierra Nevada peaked in 2016.

In [15]:
alt.Chart(high_severity, title="").mark_circle(size=80).encode(
    x=alt.X(
        "percent_of_total",
        title="%",
        axis=alt.Axis(format=''),
#         scale=alt.Scale(domain=[2012, 2021])
    ),
    y=alt.Y(
        'acre',
        title='Acres'
    ),
    color = 'year',
    tooltip=['year', 'acre', 'percent_of_total']
).properties(width=600, title='acres v.s. percent')

alt.Chart(...)

# Basal area change

In [16]:
ba_by_year, ba_by_index = analyze_index('output/sum_of_sierra_area_by_ba_index.csv')

/var/folders/zv/3yvy_fbd0mz2lvqqtd13wypc0000gp/T/ipykernel_38874/1028497440.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['percent_of_total'].iloc[k] = df.acre[k] / grouped.acre[i]
/var/folders/zv/3yvy_fbd0mz2lvqqtd13wypc0000gp/T/ipykernel_38874/1028497440.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total'].iloc[k] = grouped.acre[i]


In [17]:
ba_loss_ninety = ba_by_index[ba_by_index.ind == 7].reset_index(drop=True)

alt.Chart(ba_loss_ninety, title="Areas with more than 90% basal loss by year").mark_bar(size=20).encode(
    x=alt.X(
        "year",
        title="Year",
        axis=alt.Axis(format=''),
        scale=alt.Scale(domain=[2012, 2021])
    ),
    y=alt.Y(
        'acre',
        title='Acres',
        axis=alt.Axis(format=""),
    ),
    tooltip=['year', 'acre']
).properties(width=600)

alt.Chart(...)

Wildfire land that had lost more than 90% of its basal area totaled 501,454 acres in 2021.

In [23]:
ba_loss_ninety

,year,ind,ind_name,sqm,percent_of_total,total,acre
0,2012,7,>=90%,5.810940e+07,0.211121,6.801358e+04,14359.123287
1,2013,7,>=90%,4.219794e+08,0.332792,3.133289e+05,104273.219637
2,2014,7,>=90%,2.169261e+08,0.447116,1.198873e+05,53603.523941
3,2015,7,>=90%,2.494377e+08,0.385212,1.600088e+05,61637.302859
4,2016,7,>=90%,1.996920e+08,0.524442,9.409022e+04,49344.891660
5,2017,7,>=90%,9.534060e+07,0.253340,9.299426e+04,23559.138963
6,2018,7,>=90%,2.318256e+08,0.253152,2.262884e+05,57285.264888
7,2019,7,>=90%,1.273464e+08,0.388820,8.093180e+04,31467.932172
8,2020,7,>=90%,1.668974e+09,0.450578,9.152957e+05,412411.696718
9,2021,7,>=90%,2.029318e+09,0.441530,1.135721e+06,501454.673811


# Canopy coverage loss

In [19]:
cc_by_year, cc_by_index = analyze_index('output/sum_of_sierra_area_by_cc_index.csv')

/var/folders/zv/3yvy_fbd0mz2lvqqtd13wypc0000gp/T/ipykernel_38874/1028497440.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['percent_of_total'].iloc[k] = df.acre[k] / grouped.acre[i]
/var/folders/zv/3yvy_fbd0mz2lvqqtd13wypc0000gp/T/ipykernel_38874/1028497440.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total'].iloc[k] = grouped.acre[i]


In [20]:
cc_loss_svnfv = cc_by_index[cc_by_index.ind == 5].reset_index(drop=True)

alt.Chart(cc_loss_svnfv, title="Areas with more than 75% canopy coverage loss by year").mark_bar(size=20).encode(
    x=alt.X(
        "year",
        title="Year",
        axis=alt.Axis(format=''),
        scale=alt.Scale(domain=[2012, 2021])
    ),
    y=alt.Y(
        'acre',
        title='Acres',
        axis=alt.Axis(format=""),
    ),
    tooltip=['year', 'acre']
).properties(width=600)

alt.Chart(...)

564,010 acres of wildfire land lost more than 75% of its canopy coverage in 2021.

In [22]:
cc_loss_svnfv

,year,ind,ind_name,sqm,percent_of_total,total,acre
0,2012,5,>=75%,6.969510e+07,0.253214,6.801358e+04,17222.007686
1,2013,5,>=75%,4.820517e+08,0.380167,3.133289e+05,119117.385329
2,2014,5,>=75%,2.359170e+08,0.486259,1.198873e+05,58296.270285
3,2015,5,>=75%,2.865636e+08,0.442546,1.600088e+05,70811.298378
4,2016,5,>=75%,2.200932e+08,0.578021,9.409022e+04,54386.130186
5,2017,5,>=75%,1.102347e+08,0.292916,9.299426e+04,27239.545544
6,2018,5,>=75%,2.748825e+08,0.300169,2.262884e+05,67924.840162
7,2019,5,>=75%,1.431459e+08,0.437060,8.093180e+04,35372.067620
8,2020,5,>=75%,1.842813e+09,0.497510,9.152957e+05,455368.306365
9,2021,5,>=75%,2.282471e+09,0.496610,1.135721e+06,564010.021166
